# 🤖 SENTALIS — Sentiment Analysis
## LLM Edition (seed-2-0-mini via SumoPod AI)

**Input:** Dataset berlabel (`dataset_ml.csv`)  
**Output:** Hasil prediksi sentimen menggunakan LLM + laporan evaluasi lengkap

---

### 🗺️ Alur Notebook
```
1. Load & eksplorasi dataset
2. Analisis distribusi kelas
3. Prediksi sentimen via LLM (zero-shot / few-shot)
4. Evaluasi model — accuracy, precision, recall, F1
5. Confusion matrix & visualisasi
6. Analisis kesalahan prediksi
7. Prediksi teks baru interaktif
8. Simpan hasil
```

> 📁 Pastikan file `dataset_ml.csv` tersedia di direktori yang sama.  
> 🔑 API Key SumoPod AI sudah dikonfigurasi di Cell 1.

## 📦 Cell 1 — Install & Import

In [ ]:
!pip install openai pandas matplotlib seaborn tqdm -q

import pandas as pd
import numpy as np
import re
import json
import time
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

from openai import OpenAI

# Scikit-learn: evaluasi
from sklearn.metrics import (
    accuracy_score, classification_report,
    confusion_matrix,
    f1_score, precision_score, recall_score
)

plt.rcParams['figure.dpi'] = 120
plt.rcParams['font.family'] = 'DejaVu Sans'

WARNA = {'Negatif': '#FF3B30', 'Netral': '#8E8E93', 'Positif': '#34C759'}
LABEL_ORDER = ['Negatif', 'Netral', 'Positif']

# ─── Inisialisasi OpenAI Client dengan SumoPod AI ───
client = OpenAI(
    api_key="sk-vAAu3GrFUtzmkbPe5tAHSg",
    base_url="https://ai.sumopod.com/v1"
)

MODEL_LLM = "seed-2-0-mini-free"

print('✅ Semua library berhasil diimport!')
print(f'🤖 Model LLM: {MODEL_LLM}')

# Uji koneksi API
test_resp = client.chat.completions.create(
    model=MODEL_LLM,
    messages=[{"role": "user", "content": "Balas hanya dengan kata: OK"}],
    max_tokens=10,
    temperature=0
)
print(f'✅ Koneksi API berhasil: {test_resp.choices[0].message.content.strip()}')

## 📂 Cell 2 — Load Dataset

In [ ]:
import os

USE_PRESPLIT = os.path.exists('dataset_train.csv') and os.path.exists('dataset_test.csv')

if USE_PRESPLIT:
    df_train_raw = pd.read_csv('dataset_train.csv', encoding='utf-8-sig')
    df_test_raw  = pd.read_csv('dataset_test.csv',  encoding='utf-8-sig')
    print(f'✅ Menggunakan pre-split dataset')
    print(f'   Train: {len(df_train_raw)} | Test: {len(df_test_raw)}')
else:
    for enc in ['utf-8-sig', 'utf-8', 'latin-1']:
        try:
            df_full = pd.read_csv('dataset_ml.csv', encoding=enc)
            print(f'✅ Loaded dataset_ml.csv ({enc})')
            break
        except Exception:
            continue

def normalize_df(df):
    df = df.copy()
    df['teks_bersih'] = df['teks_bersih'].fillna('').astype(str).str.strip()
    df = df[df['teks_bersih'].str.len() > 0].reset_index(drop=True)
    return df

if USE_PRESPLIT:
    df_train_raw = normalize_df(df_train_raw)
    df_test_raw  = normalize_df(df_test_raw)
    df_full = pd.concat([df_train_raw, df_test_raw], ignore_index=True)
else:
    df_full = normalize_df(df_full)

print(f'\n📊 Total data: {len(df_full)}')
print(f'📋 Kolom    : {list(df_full.columns)}')
display(df_full[['teks_bersih', 'label']].head(6))

## 🔍 Cell 3 — Eksplorasi & Analisis Distribusi Kelas

In [ ]:
dist = df_full['label'].value_counts()
total = len(df_full)

print('=' * 50)
print('📊 DISTRIBUSI KELAS')
print('=' * 50)
for label in LABEL_ORDER:
    n = dist.get(label, 0)
    pct = n / total * 100
    bar = '█' * int(pct / 2.5)
    print(f'  {label:10s} {bar:30s} {n:4d} ({pct:.1f}%)')
print(f'  Total     {" " * 30} {total}')

n_max = dist.max()
n_min = dist.min()
ratio = n_max / n_min
print(f'\n⚠️  Imbalance Ratio: {ratio:.1f}x')

if ratio > 5:
    print('\n❗ Dataset SANGAT TIDAK SEIMBANG.')
    print('   Catatan: LLM zero-shot tidak terpengaruh class imbalance secara langsung.')
elif ratio > 2:
    print('\n⚡ Dataset cukup tidak seimbang.')
else:
    print('\n✅ Dataset cukup seimbang.')

df_full['n_kata'] = df_full['teks_bersih'].str.split().str.len()
print('\n─── Rata-rata panjang teks per kelas ───')
for label in LABEL_ORDER:
    sub = df_full[df_full['label'] == label]['n_kata']
    if len(sub) > 0:
        print(f'  {label:10s}: rata {sub.mean():.1f} kata, max {sub.max()} kata')

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
fig.patch.set_facecolor('#F2F2F7')

ax = axes[0]
ax.set_facecolor('white')
bars = ax.bar(LABEL_ORDER,
              [dist.get(l, 0) for l in LABEL_ORDER],
              color=[WARNA[l] for l in LABEL_ORDER],
              edgecolor='white', width=0.55)
for bar, label in zip(bars, LABEL_ORDER):
    h = bar.get_height()
    pct = h / total * 100
    ax.text(bar.get_x() + bar.get_width() / 2, h + 2,
            f'{int(h)}\n({pct:.1f}%)', ha='center', fontsize=10, fontweight='bold')
ax.set_title('Distribusi Kelas', fontweight='bold', fontsize=13)
ax.set_ylabel('Jumlah Sampel')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

ax2 = axes[1]
ax2.set_facecolor('white')
data_box = [df_full[df_full['label'] == l]['n_kata'].values for l in LABEL_ORDER]
bp = ax2.boxplot(data_box, patch_artist=True, labels=LABEL_ORDER,
                 medianprops=dict(color='white', linewidth=2))
for patch, label in zip(bp['boxes'], LABEL_ORDER):
    patch.set_facecolor(WARNA[label])
    patch.set_alpha(0.8)
ax2.set_title('Sebaran Panjang Teks (kata)', fontweight='bold', fontsize=13)
ax2.set_ylabel('Jumlah Kata')
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)

plt.tight_layout(pad=2)
plt.savefig('llm_distribusi_kelas.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## ✂️ Cell 4 — Train / Test Split

> Untuk LLM zero-shot, kita **tidak perlu** data training.  
> Namun tetap melakukan split agar evaluasi konsisten dengan baseline ML.  
> Beberapa contoh dari data train digunakan sebagai **few-shot examples** dalam prompt.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_full['teks_bersih'].values
y = df_full['label'].values

if USE_PRESPLIT:
    X_train = df_train_raw['teks_bersih'].values
    y_train = df_train_raw['label'].values
    X_test  = df_test_raw['teks_bersih'].values
    y_test  = df_test_raw['label'].values
    print('✅ Menggunakan pre-split (dari file train/test terpisah)')
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    print('✅ Stratified split 80/20 selesai')

print(f'\n   Train : {len(X_train)} sampel (digunakan sebagai few-shot pool)')
print(f'   Test  : {len(X_test)} sampel (dievaluasi oleh LLM)')

# Ambil few-shot examples: 2 per kelas dari data train
FEW_SHOT_N = 2
few_shot_examples = []
for label in LABEL_ORDER:
    idx = np.where(y_train == label)[0]
    # Pilih contoh dengan panjang sedang (tidak terlalu pendek/panjang)
    lengths = np.array([len(X_train[i].split()) for i in idx])
    med = np.median(lengths)
    sorted_idx = idx[np.argsort(np.abs(lengths - med))]
    for i in sorted_idx[:FEW_SHOT_N]:
        few_shot_examples.append({'teks': X_train[i], 'label': label})

print(f'\n📚 Few-shot examples ({FEW_SHOT_N} per kelas):')
for ex in few_shot_examples:
    icon = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}[ex['label']]
    print(f'   {icon} [{ex["label"]}] "{ex["teks"][:60]}"')

## 🤖 Cell 5 — Desain Prompt & Fungsi Prediksi LLM

Kita gunakan pendekatan **few-shot prompting** agar LLM lebih memahami konteks dataset  
(komentar warga tentang pemerintah/infrastruktur daerah dalam Bahasa Indonesia/Sunda).

In [ ]:
# ─── Buat System Prompt ───
SYSTEM_PROMPT = """Kamu adalah sistem analisis sentimen untuk komentar warga mengenai layanan pemerintah daerah, infrastruktur, dan kebijakan publik dalam Bahasa Indonesia dan Bahasa Sunda.

Tugasmu adalah mengklasifikasikan sentimen komentar menjadi salah satu dari tiga kategori:
- Negatif: komentar yang mengungkapkan ketidakpuasan, keluhan, kritik, atau kekecewaan
- Netral: komentar yang bersifat informatif, pertanyaan, atau tidak jelas arah sentimennya
- Positif: komentar yang mengungkapkan kepuasan, apresiasi, pujian, atau dukungan

Aturan:
1. Jawab HANYA dengan satu kata: Negatif, Netral, atau Positif
2. Jangan tambahkan penjelasan, tanda baca, atau kata lain apapun
3. Perhatikan konteks lokal dan ungkapan dalam Bahasa Sunda"""

# ─── Buat Few-Shot Messages ───
def build_few_shot_messages(examples):
    """Buat daftar few-shot message dari contoh berlabel."""
    messages = []
    for ex in examples:
        messages.append({"role": "user", "content": ex['teks']})
        messages.append({"role": "assistant", "content": ex['label']})
    return messages

FEW_SHOT_MESSAGES = build_few_shot_messages(few_shot_examples)

# ─── Fungsi Prediksi LLM ───
def predict_llm(teks, max_retries=3, delay=1.0):
    """
    Prediksi sentimen satu teks menggunakan LLM.
    Returns: label string ('Negatif', 'Netral', 'Positif') atau None jika gagal
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        *FEW_SHOT_MESSAGES,
        {"role": "user", "content": str(teks).strip()}
    ]

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model=MODEL_LLM,
                messages=messages,
                max_tokens=10,
                temperature=0  # deterministic
            )
            raw = response.choices[0].message.content.strip()

            # Normalisasi output
            raw_lower = raw.lower()
            if 'negatif' in raw_lower or 'negative' in raw_lower:
                return 'Negatif'
            elif 'positif' in raw_lower or 'positive' in raw_lower:
                return 'Positif'
            elif 'netral' in raw_lower or 'neutral' in raw_lower:
                return 'Netral'
            else:
                # Fallback: cek apakah persis salah satu label
                for label in LABEL_ORDER:
                    if raw.strip() == label:
                        return label
                # Jika tidak dikenali, default Netral
                print(f'  ⚠️  Output tidak dikenali: "{raw}" → default Netral')
                return 'Netral'

        except Exception as e:
            if attempt < max_retries - 1:
                print(f'  ⚠️  Retry {attempt+1}/{max_retries}: {e}')
                time.sleep(delay * (attempt + 1))
            else:
                print(f'  ❌ Gagal setelah {max_retries} percobaan: {e}')
                return None

# Test fungsi prediksi
print('🧪 Test prediksi LLM:')
contoh_test = [
    ('Jalanan rusak parah, sudah bertahun-tahun tidak diperbaiki!', 'Negatif'),
    ('Terima kasih pak bupati, programnya sangat membantu warga.', 'Positif'),
    ('Kapan pendaftaran bantuan sosial dibuka?', 'Netral'),
]
for teks, expected in contoh_test:
    pred = predict_llm(teks)
    icon_pred = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}.get(pred, '⚫')
    icon_exp  = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}.get(expected, '⚫')
    status = '✅' if pred == expected else '❌'
    print(f'  {status} Pred: {icon_pred} {pred:8s} | Expected: {icon_exp} {expected:8s} | "{teks[:55]}"')

print(f'\n✅ Fungsi prediksi LLM siap digunakan!')

## 🔄 Cell 6 — Jalankan Prediksi pada Test Set

> ⚠️ **Perhatian:** Cell ini akan memanggil API sebanyak `len(X_test)` kali.  
> Untuk dataset besar, pertimbangkan untuk menetapkan `MAX_SAMPLES` agar lebih cepat.  
> Set `MAX_SAMPLES = None` untuk memproses semua data.

In [ ]:
# ─── Konfigurasi ───
MAX_SAMPLES = 200   # Set None untuk semua data, atau integer untuk subset
DELAY_BETWEEN_CALLS = 0.3  # detik antar API call (hindari rate limit)

if MAX_SAMPLES is not None and MAX_SAMPLES < len(X_test):
    # Stratified sampling agar distribusi label terjaga
    from sklearn.model_selection import train_test_split as tts
    idx_sample = []
    for label in LABEL_ORDER:
        idx_label = np.where(y_test == label)[0]
        n_take = max(1, int(MAX_SAMPLES * len(idx_label) / len(y_test)))
        idx_sample.extend(np.random.choice(idx_label, min(n_take, len(idx_label)), replace=False))
    idx_sample = sorted(idx_sample)
    X_eval = X_test[idx_sample]
    y_eval = y_test[idx_sample]
    print(f'📋 Evaluasi subset: {len(X_eval)} sampel dari {len(X_test)} total')
else:
    X_eval = X_test
    y_eval = y_test
    print(f'📋 Evaluasi semua test data: {len(X_eval)} sampel')

print(f'\n🚀 Mulai prediksi LLM...')
print(f'   Estimasi waktu: ~{len(X_eval) * DELAY_BETWEEN_CALLS / 60:.1f} menit')
print()

# ─── Loop Prediksi ───
y_pred_llm = []
failed_idx = []

for i, teks in enumerate(tqdm(X_eval, desc='Prediksi LLM')):
    pred = predict_llm(teks)
    if pred is None:
        pred = 'Netral'  # default jika gagal
        failed_idx.append(i)
    y_pred_llm.append(pred)
    if DELAY_BETWEEN_CALLS > 0:
        time.sleep(DELAY_BETWEEN_CALLS)

y_pred_llm = np.array(y_pred_llm)

print(f'\n✅ Prediksi selesai!')
print(f'   Total: {len(y_pred_llm)}')
print(f'   Gagal/fallback: {len(failed_idx)}')

# Simpan hasil sementara
df_hasil = pd.DataFrame({
    'teks': X_eval,
    'label_aktual': y_eval,
    'pred_llm': y_pred_llm,
})
df_hasil['benar'] = df_hasil['label_aktual'] == df_hasil['pred_llm']

print(f'\n📈 Quick result:')
acc_quick = accuracy_score(y_eval, y_pred_llm)
f1_quick  = f1_score(y_eval, y_pred_llm, average='macro', zero_division=0)
print(f'   Accuracy : {acc_quick:.4f}')
print(f'   Macro F1 : {f1_quick:.4f}')

## 📋 Cell 7 — Evaluasi Detail & Classification Report

In [ ]:
print('=' * 60)
print('📊 CLASSIFICATION REPORT — LLM (seed-2-0-mini)')
print('=' * 60)
print(classification_report(y_eval, y_pred_llm, target_names=LABEL_ORDER, zero_division=0))

precision = precision_score(y_eval, y_pred_llm, labels=LABEL_ORDER, average=None, zero_division=0)
recall    = recall_score(y_eval, y_pred_llm, labels=LABEL_ORDER, average=None, zero_division=0)
f1        = f1_score(y_eval, y_pred_llm, labels=LABEL_ORDER, average=None, zero_division=0)
support   = [np.sum(y_eval == l) for l in LABEL_ORDER]

df_report = pd.DataFrame({
    'Kelas':     LABEL_ORDER,
    'Precision': [f'{p:.3f}' for p in precision],
    'Recall':    [f'{r:.3f}' for r in recall],
    'F1-Score':  [f'{f:.3f}' for f in f1],
    'Support':   support,
})

macro_f1 = f1_score(y_eval, y_pred_llm, average='macro', zero_division=0)
acc      = accuracy_score(y_eval, y_pred_llm)

print(f'\n📈 Ringkasan:')
print(f'   Accuracy       : {acc:.4f} ({acc*100:.2f}%)')
print(f'   Macro F1       : {macro_f1:.4f}')
print(f'   Weighted F1    : {f1_score(y_eval, y_pred_llm, average="weighted", zero_division=0):.4f}')
print()
display(df_report)

# Visualisasi metrik per kelas
fig, ax = plt.subplots(figsize=(10, 4))
fig.patch.set_facecolor('#F2F2F7')
ax.set_facecolor('white')

x = np.arange(len(LABEL_ORDER))
w = 0.25
metrics_vals = [precision, recall, f1]
metric_names = ['Precision', 'Recall', 'F1-Score']
m_colors     = ['#007AFF', '#5856D6', '#34C759']

for i, (vals, name, color) in enumerate(zip(metrics_vals, metric_names, m_colors)):
    bars = ax.bar(x + i * w, vals, w, label=name, color=color, alpha=0.85, edgecolor='white')
    for bar, val in zip(bars, vals):
        if val > 0:
            ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.01,
                    f'{val:.2f}', ha='center', va='bottom', fontsize=8, fontweight='bold')

ax.set_xticks(x + w)
ax.set_xticklabels(LABEL_ORDER, fontsize=11)
ax.set_ylim(0, 1.15)
ax.set_ylabel('Score')
ax.set_title('Precision / Recall / F1 per Kelas — LLM (seed-2-0-mini)',
             fontweight='bold', fontsize=13)
ax.legend(fontsize=10)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.axhline(y=macro_f1, color='gray', linestyle='--', linewidth=1, alpha=0.6)

plt.tight_layout()
plt.savefig('llm_metrik_kelas.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

## 🟦 Cell 8 — Confusion Matrix

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
fig.patch.set_facecolor('#F2F2F7')
ax.set_facecolor('white')

cm = confusion_matrix(y_eval, y_pred_llm, labels=LABEL_ORDER)
cm_pct = cm.astype(float) / cm.sum(axis=1, keepdims=True) * 100

annot = np.array(
    [[f'{cm[i,j]}\n({cm_pct[i,j]:.0f}%)' for j in range(len(LABEL_ORDER))]
     for i in range(len(LABEL_ORDER))]
)

sns.heatmap(
    cm_pct, annot=annot, fmt='',
    xticklabels=LABEL_ORDER, yticklabels=LABEL_ORDER,
    cmap='Blues', ax=ax,
    linewidths=0.5, linecolor='#F2F2F7',
    cbar_kws={'shrink': 0.8}
)
ax.set_title(f'Confusion Matrix — LLM\nAccuracy: {acc:.4f}', fontweight='bold', fontsize=12)
ax.set_xlabel('Prediksi', fontsize=11)
ax.set_ylabel('Aktual', fontsize=11)

plt.tight_layout()
plt.savefig('llm_confusion_matrix.png', dpi=150, bbox_inches='tight', facecolor='#F2F2F7')
plt.show()

print('\n📝 Interpretasi Confusion Matrix:')
for i, label in enumerate(LABEL_ORDER):
    row_sum = cm[i].sum()
    correct = cm[i, i]
    if row_sum > 0:
        print(f'   {label:10s}: {correct}/{row_sum} benar ({correct/row_sum*100:.1f}% recall)')

## ❌ Cell 9 — Analisis Kesalahan Prediksi

In [ ]:
df_salah = df_hasil[~df_hasil['benar']].copy()

print(f'✅ Prediksi benar : {df_hasil["benar"].sum()} / {len(df_hasil)}')
print(f'❌ Prediksi salah : {(~df_hasil["benar"]).sum()} / {len(df_hasil)}')

if len(df_salah) > 0:
    print(f'\n─── Pola Kesalahan ───')
    pola = df_salah.groupby(['label_aktual', 'pred_llm']).size().reset_index(name='count')
    pola = pola.sort_values('count', ascending=False)
    for _, row in pola.iterrows():
        print(f'   {row["label_aktual"]:10s} → diprediksi {row["pred_llm"]:10s}: {row["count"]} kasus')

    print(f'\n─── 10 Contoh Kesalahan Prediksi ───')
    for _, row in df_salah.head(10).iterrows():
        teks_preview = row['teks'][:65] + '...' if len(row['teks']) > 65 else row['teks']
        print(f'   Aktual: {row["label_aktual"]:10s} | Prediksi: {row["pred_llm"]:10s}')
        print(f'   Teks  : "{teks_preview}"')
        print()

# Export hasil prediksi
df_hasil.to_csv('hasil_prediksi_llm.csv', index=False, encoding='utf-8-sig')
print('💾 Tersimpan: hasil_prediksi_llm.csv')

## 🧪 Cell 10 — Prediksi Teks Baru Interaktif

In [ ]:
def analisis_sentimen(teks_input):
    """
    Prediksi sentimen teks baru dengan tampilan detail.
    Untuk LLM, kita juga meminta penjelasan singkat.
    """
    if not teks_input or not teks_input.strip():
        print('⚠️  Teks kosong!')
        return

    # Prediksi label
    label = predict_llm(teks_input)

    # Minta penjelasan singkat
    messages_explain = [
        {"role": "system", "content": "Kamu adalah analis sentimen. Berikan penjelasan singkat (1-2 kalimat) mengapa teks berikut termasuk sentimen yang disebutkan."},
        {"role": "user", "content": f'Teks: "{teks_input}"\nSentimen: {label}\nJelaskan singkat:'}
    ]
    try:
        resp_explain = client.chat.completions.create(
            model=MODEL_LLM,
            messages=messages_explain,
            max_tokens=150,
            temperature=0.3
        )
        alasan = resp_explain.choices[0].message.content.strip()
    except:
        alasan = '(penjelasan tidak tersedia)'

    icon = {'Negatif': '🔴', 'Netral': '⚪', 'Positif': '🟢'}.get(label, '⚫')

    print(f'  Input     : "{teks_input[:70]}"')
    print(f'  Prediksi  : {icon} {label}')
    print(f'  Alasan    : {alasan}')
    print()

    return {'label': label, 'alasan': alasan}


# ─── Uji coba prediksi ───
teks_uji = [
    'Alhamdulillah jalan depan rumah akhirnya diperbaiki, terima kasih pak bupati!',
    'Jalan masih rusak parah, kapan diperbaiki? Sudah bertahun-tahun begini.',
    'Cukup bagus kepemimpinan bapa bupati, semoga terus maju.',
    'Ga ngaruh apa2. Sama aja bohong. Hahaha.',
    'Hmm biasa aja sih, gak ada yang spesial.',
    'Pengangguran masih tinggi, lapangan kerja kurang, galian pasir masih beroperasi.',
]

print('=' * 60)
print('🧪 PREDIKSI TEKS BARU')
print('=' * 60)
for teks in teks_uji:
    analisis_sentimen(teks)

## 💾 Cell 11 — Simpan Hasil & Laporan

In [ ]:
import zipfile

# Simpan metadata & hasil evaluasi
model_info = {
    'nama_model':    MODEL_LLM,
    'provider':      'SumoPod AI (OpenAI-compatible)',
    'metode':        'Few-shot prompting',
    'few_shot_n':    FEW_SHOT_N,
    'n_eval':        len(X_eval),
    'hasil_evaluasi': {
        'accuracy':   round(float(acc), 4),
        'macro_f1':   round(float(macro_f1), 4),
        'per_kelas': {
            label: {
                'precision': round(float(precision_score(y_eval, y_pred_llm, labels=[label], average='macro', zero_division=0)), 4),
                'recall':    round(float(recall_score(y_eval, y_pred_llm, labels=[label], average='macro', zero_division=0)), 4),
                'f1':        round(float(f1_score(y_eval, y_pred_llm, labels=[label], average='macro', zero_division=0)), 4),
                'support':   int(np.sum(y_eval == label)),
            } for label in LABEL_ORDER
        },
    },
    'label_map':     {'Negatif': 0, 'Netral': 1, 'Positif': 2},
    'cara_pakai': [
        "from openai import OpenAI",
        "client = OpenAI(api_key='sk-...', base_url='https://ai.sumopod.com/v1')",
        "label = predict_llm(teks)  # gunakan fungsi predict_llm()",
    ]
}

with open('llm_model_info.json', 'w', encoding='utf-8') as f:
    json.dump(model_info, f, ensure_ascii=False, indent=2)
print('✅ Tersimpan: llm_model_info.json')

# ZIP semua output
output_files = [
    'hasil_prediksi_llm.csv', 'llm_model_info.json',
    'llm_distribusi_kelas.png', 'llm_metrik_kelas.png', 'llm_confusion_matrix.png',
]

with zipfile.ZipFile('SENTALIS_LLM_output.zip', 'w', zipfile.ZIP_DEFLATED) as zf:
    for fname in output_files:
        if os.path.exists(fname):
            zf.write(fname)
            print(f'  + {fname}')

print(f'\n📦 ZIP: SENTALIS_LLM_output.zip ({os.path.getsize("SENTALIS_LLM_output.zip")/1024:.1f} KB)')

try:
    from google.colab import files
    files.download('SENTALIS_LLM_output.zip')
    print('⬇️  Download dimulai...')
except ImportError:
    print('📂 File tersimpan di direktori saat ini.')

---

## 📊 Ringkasan & Catatan Penting

### Keunggulan LLM vs TF-IDF + Naive Bayes

| Aspek | TF-IDF + NB | LLM (seed-2-0-mini) |
|---|---|---|
| **Training data** | Diperlukan | ❌ Tidak perlu (zero/few-shot) |
| **Pemahaman konteks** | Terbatas | ✅ Lebih dalam |
| **Bahasa Sunda/slang** | Bergantung kosakata | ✅ Lebih robust |
| **Imbalanced data** | Bermasalah | ✅ Tidak terpengaruh |
| **Kecepatan** | ✅ Sangat cepat | ❌ Lambat (API call per data) |
| **Biaya** | ✅ Gratis | Tergantung API pricing |
| **Interpretasi** | Bisa lihat fitur | ❌ Black box |

### Tips Meningkatkan Akurasi LLM

Untuk mendapatkan hasil yang lebih baik, Anda dapat mencoba:
- Menambah jumlah few-shot examples (dari 2 menjadi 5 per kelas)
- Memperkaya system prompt dengan penjelasan konteks yang lebih detail
- Menggunakan model yang lebih besar (misal `seed-2-0` non-free)
- Menerapkan chain-of-thought: minta LLM jelaskan dulu, baru beri label

### Cara Pakai Fungsi Prediksi

```python
from openai import OpenAI

client = OpenAI(
    api_key="sk-vAAu3GrFUtzmkbPe5tAHSg",
    base_url="https://ai.sumopod.com/v1"
)

teks = "Jalan masih rusak dan gelap!"
label = predict_llm(teks)  # 'Negatif'
```

---
*SENTALIS — LLM Edition*  
*Few-shot Sentiment Analysis menggunakan seed-2-0-mini via SumoPod AI*